In [2]:

import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

from sklearn.metrics import SCORERS
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate


from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

from sklearn import preprocessing

from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from xgboost.sklearn import XGBClassifier



In [3]:
tf = pd.read_csv('featuresMagui.csv') #mis features 
tf_seba = pd.read_csv('features_seba.csv') #los features de Seba
tf_santi = pd.read_csv('santi_timefeatures.csv')
tf_santi_2 = pd.read_csv('Santi_FeaturesConEventos.csv')
labels = pd.read_csv('../data/labels_training_set.csv') #las personas de las cuales tengo Info
personas =pd.read_csv('../data/trocafone_kaggle_test.csv') #las personas a las que le tengo que predecir

In [4]:
#Datos a entregar
datos = pd.merge(tf, tf_seba, on = 'person', how = 'inner')
datos = pd.merge(datos, tf_santi, on = 'person', how = 'inner')
datos = pd.merge(datos, tf_santi_2, on = 'person', how = 'inner')
datos = pd.merge(personas, datos, on = 'person', how = 'inner')

In [5]:
#Datos para entrenar
labels_f = pd.merge(tf,tf_seba, on = 'person',how = 'inner')
labels_f = pd.merge(labels_f,tf_santi, on = 'person',how = 'inner')
labels_f = pd.merge(labels_f,tf_santi_2, on = 'person',how = 'inner')
labels_f = pd.merge(labels, labels_f, on = 'person', how = 'inner')

In [53]:

y = labels_f.iloc[:,1:2]
X = labels_f.iloc[:,3: 406]

In [54]:
X = X.drop('device_type', axis = 1)

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.1, random_state= 123)

In [56]:
rus= RandomUnderSampler(return_indices=True)

#id_rus son los índices
#X_train, y_train, id_rus = rus.fit_sample(X_train, y_train)

In [57]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

In [58]:
clf.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [59]:
preds = clf.predict(X_test)
preds_prob = clf.predict_proba(X_test)[:,1]
train_accuracy = accuracy_score(y_train, clf.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds_prob)
matriz_de_confusion = confusion_matrix(y_test, preds)
print('train acurracy: ')
print(train_accuracy)
print('test acurracy: ')
print(test_accuracy)
print('Matriz de confusión: ')
print(matriz_de_confusion)
print('Área bajo la curva: ')
print(area_debajo_de_curva)

train acurracy: 
0.949290293040293
test acurracy: 
0.9515962924819773
Matriz de confusión: 
[[1848    0]
 [  94    0]]
Área bajo la curva: 
0.8260482868195633


In [62]:
importancia = clf.feature_importances_
len(importancia)
#for columna in datos

402

In [67]:
importancia_con_nombre = {}
features = X.columns
for i in range(402):
    importancia_con_nombre[features[i]] = importancia[i]
importancia_con_nombre

{'esNuevo': 0.0,
 'cantidadDeVisitas': 0.01098474513870992,
 'ingresoPorPublicidad': 0.0,
 'Facebook': 0.0,
 'FacebookAds': 0.00122458573014752,
 'FacebookSocial': 0.0,
 'MARKETING SOCIAL': 0.0,
 'afiliado': 0.0,
 'afilio': 0.0,
 'bing': 0.0,
 'blog': 0.0,
 'buscape': 0.0,
 'criteo': 0.0,
 'datacrush': 0.0,
 'emblue': 0.0,
 'gizmodo': 0.0,
 'google': 0.0,
 'indexa': 0.0,
 'manifest': 0.0,
 'mercadopago': 0.0,
 'onsite': 0.0,
 'rakuten': 0.0,
 'rtbhouse': 0.0,
 'voxus': 0.0,
 'yotpo': 0.0,
 'zanox': 0.0,
 'deviceSmartphone': 0.0019158268269958694,
 'deviceComputer': 0.0,
 'deviceTablet': 0.0,
 'deviceUnknown': 0.0,
 'cantVisitasAVP': 0.005198710343276267,
 'maniana': 0.0,
 'tarde': 0.0,
 'noche': 0.02055015930364258,
 'madrugada': 0.0,
 'cantDirect': 0.008651389548144945,
 'cantEmail': 0.0,
 'cantOrganic': 0.0,
 'cantPaid': 0.0,
 'cantReferral': 0.0016158698572296718,
 'cantSocial': 0.0,
 'cantUnknown': 0.0,
 'cantVisitasDomingo': 0.00044824231656681325,
 'cantVisitasLunes': 0.0,
 'cant

In [64]:
atributos_importantes = []
for clave in importancia_con_nombre.keys():
    if importancia_con_nombre[clave] > 0:
        atributos_importantes.append(clave)

In [65]:
atributos_importantes

['cantidadDeVisitas',
 'FacebookAds',
 'deviceSmartphone',
 'cantVisitasAVP',
 'noche',
 'cantDirect',
 'cantReferral',
 'cantVisitasDomingo',
 'cantVisitasMiercoles',
 'cantidadDeEventos',
 'cantBandListing',
 'cantCheckout',
 'cantStaticpage',
 'cantViewedProduct',
 'cantVisitedSite',
 'checkout_x',
 'searchEngineHit',
 'brand listing mes 5',
 'checkout mes 5',
 'viewed product mes 5',
 'visited site mes 5',
 'dias_hasta_ultimo',
 'ad campaign hit mes 4',
 'brand listing mes 4',
 'generic listing mes 4',
 'search engine hit mes 4',
 'viewed product mes 4',
 'visited site mes 4',
 'distan_dias',
 'nuevo_mes_5',
 'cant_dias_dist',
 'modelos_dist',
 'promedio_por_mes',
 'mes_primer_entrada',
 'promedio_por_dia_y',
 'mes_3',
 'mes_4',
 'mes_5',
 'dia_2',
 'dia_3',
 'dia_5',
 'dia_7',
 'dia_8',
 'dia_9',
 'dia_25',
 'dia_28',
 'dia_30',
 'dia_31',
 'primer_quincena',
 'segunda_quincena',
 'Friday',
 'Thursday',
 'Tuesday',
 'fin_de_semana',
 'martes_miercoles_jueves',
 'hora_10',
 'hora_1

In [68]:
def xgboost(X_p, y_p, X_train_p, X_test_p, y_train_p, y_test_p):
    xg_reg = xgb.XGBClassifier(learning_rate =0.05,n_estimators=1000,max_depth=5,min_child_weight=1,gamma=0,
                           subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,
                            seed=27)
    xg_reg.fit(X_train_p,y_train_p, early_stopping_rounds = 5, eval_set=[(X_test, y_test)])
    preds = xg_reg.predict(X_test_p)
    preds_prob = xg_reg.predict_proba(X_test_p)[:,1]
    train_accuracy = accuracy_score(y_train_p, xg_reg.predict(X_train_p))
    test_accuracy = accuracy_score(y_test_p, preds)
    area_debajo_de_curva = roc_auc_score(y_test_p, preds_prob)
    matriz_de_confusion = confusion_matrix(y_test_p, preds)
    print('train acurracy: ')
    print(train_accuracy)
    print('test acurracy: ')
    print(test_accuracy)
    print('Matriz de confusión: ')
    print(matriz_de_confusion)
    print('Área bajo la curva: ')
    print(area_debajo_de_curva)
    return(xg_reg)

In [69]:
xg_reg =xgboost(X, y, X_train, X_test, y_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-error:0.053038
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.053038
[2]	validation_0-error:0.049949
[3]	validation_0-error:0.048919
[4]	validation_0-error:0.049434
[5]	validation_0-error:0.049434
[6]	validation_0-error:0.049434
[7]	validation_0-error:0.049434
[8]	validation_0-error:0.048404
[9]	validation_0-error:0.047889
[10]	validation_0-error:0.048404
[11]	validation_0-error:0.048404
[12]	validation_0-error:0.048404
[13]	validation_0-error:0.048404
[14]	validation_0-error:0.049434
Stopping. Best iteration:
[9]	validation_0-error:0.047889

train acurracy: 
0.9537545787545788
test acurracy: 
0.9521112255406797
Matriz de confusión: 
[[1845    3]
 [  90    4]]
Área bajo la curva: 
0.8488273694390717


In [72]:
scaler = preprocessing.StandardScaler().fit(X)
X_transformed = scaler.transform(X)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  


In [73]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size= 0.1, random_state= 123)

xg_reg =xgboost(X_transformed, y, X_train, X_test, y_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0]	validation_0-error:0.053038
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.053038
[2]	validation_0-error:0.049949
[3]	validation_0-error:0.048919
[4]	validation_0-error:0.049434
[5]	validation_0-error:0.049434
[6]	validation_0-error:0.049434
[7]	validation_0-error:0.049434
[8]	validation_0-error:0.048404
[9]	validation_0-error:0.047889
[10]	validation_0-error:0.048404
[11]	validation_0-error:0.048404
[12]	validation_0-error:0.048404
[13]	validation_0-error:0.048404
[14]	validation_0-error:0.049434
Stopping. Best iteration:
[9]	validation_0-error:0.047889

train acurracy: 
0.9537545787545788
test acurracy: 
0.9521112255406797
Matriz de confusión: 
[[1845    3]
 [  90    4]]
Área bajo la curva: 
0.8488273694390717


In [74]:
rus= RandomUnderSampler(return_indices=True)

#id_rus son los índices
X_rus_train, y_rus_train, id_rus = rus.fit_sample(X_train, y_train)
print(X_rus_train)


xg_reg = xgboost(X_transformed, y ,X_rus_train, X_test, y_rus_train, y_test)

[[ 1.27472836 -0.41485294 -2.12272366 ... -0.06202043 -0.03261667
  -0.02006217]
 [ 1.27472836 -0.41485294 -2.12272366 ... -0.06202043 -0.03261667
  -0.02006217]
 [-0.78448086 -0.2240503   0.47109288 ... -0.06202043 -0.03261667
  -0.02006217]
 ...
 [-0.78448086  0.44375894  0.47109288 ... -0.06202043 -0.03261667
  -0.02006217]
 [-0.78448086  0.06215366  0.47109288 ... -0.06202043 -0.03261667
  -0.02006217]
 [-0.78448086  0.53916026  0.47109288 ... -0.06202043 -0.03261667
  -0.02006217]]
[0]	validation_0-error:0.216272
Will train until validation_0-error hasn't improved in 5 rounds.


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1]	validation_0-error:0.226056
[2]	validation_0-error:0.213697
[3]	validation_0-error:0.213697
[4]	validation_0-error:0.214727
[5]	validation_0-error:0.217817
[6]	validation_0-error:0.209578
[7]	validation_0-error:0.209063
[8]	validation_0-error:0.212667
[9]	validation_0-error:0.210608
[10]	validation_0-error:0.205458
[11]	validation_0-error:0.207518
[12]	validation_0-error:0.207518
[13]	validation_0-error:0.210093
[14]	validation_0-error:0.211637
[15]	validation_0-error:0.210608
Stopping. Best iteration:
[10]	validation_0-error:0.205458

train acurracy: 
0.837471783295711
test acurracy: 
0.7945417095777549
Matriz de confusión: 
[[1474  374]
 [  25   69]]
Área bajo la curva: 
0.840330547112462
